In [2]:
%pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import random
import time
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import PairwiseAligner
from Bio.Blast import NCBIWWW, NCBIXML

def random_deletion(seq, num_deletions):
    seq = list(seq)
    for _ in range(num_deletions):
        del seq[random.randint(0, len(seq) - 1)]
    return ''.join(seq)

def random_insertion(seq, num_insertions):
    seq = list(seq)
    for _ in range(num_insertions):
        seq.insert(random.randint(0, len(seq) - 1), random.choice("ACGT"))
    return ''.join(seq)

def random_mutation(seq, num_mutations):
    seq = list(seq)
    for _ in range(num_mutations):
        seq[random.randint(0, len(seq) - 1)] = random.choice("ACGT")
    return ''.join(seq)

def repeat_subsequence(seq, start, length):
    subseq = seq[start:start + length]
    return seq[:start + length] + subseq + seq[start + length:]

def delete_subsequence(seq, start, length):
    return seq[:start] + seq[start + length:]

original_seq = """ATATATATATATTTTTTTATATACATTATATATTTATATATAAATAAAATTTTTATTACATATATACATT
ATATACTTATACATAATTTTTTATTTTATTATATATTTATATATCTCTATTATATACATTATATAATTAT
ATATCATATATTTATATATGATATAATTATATATCATATATTTATATATGATATAATTATATATCATATA
TTTATATATGATATAATTATATATCATATATTTATATATGATATAATATATATGTATATAATATATAAAT
ATATATAAATATATAATATATATTTATATATATTTTATATATATACACATATTATATATGTGTATATTAT
ATAATATATACACATATTATATATGTGTATATTATATAATATACACATATATCATATAATTATATATTAT
ATAATATATATATTATATATGTATATATTATATATTATATAATGTATATATTATATATTATATAATGTAT
ATATTATATCATATATAATATATTATATATATGTATATATTATATATGATATATAATATATAATATATAT
GTATATATTATATATGATATAATATATACATATTATATATGTATATATTATATATGATATAATATATACA
TATTATATATGTATATATTATATATGATATAATATATACATATTATATGTATATATTATATAATATATAC
ATATATATGTATATATTATATATGTATATATTTATATACATATATAATATATGTATATATTATGTATATA
TTTATATACATATATTATATATAATATATATAATATATACGTATATAATATATAATACACTATATAATAT
ATACGTATATAATATATATATTAGATATAATATATACGTATATAATATATAATACACTATATAATATATA
CGTATATAATACATATATACTATATAATATATACGTATATAATACATATATACCATATAATATATACGTA
TATAATACATGTATTATATACGTATATAATATATATGTATACAATATATACGTATATAATATATATGTAT
ATATTATATAAATTATATATTATATATATGTATTATATAAATTATATATTTTTT"""
X = random_deletion(original_seq, len(original_seq) - 400)

def generate_variations(X, k):
    X1 = random_insertion(X, k)
    X1 = random_deletion(X1, k)
    X1 = random_mutation(X1, k)
    X1 = repeat_subsequence(X1, 2 * k, k)
    X1 = delete_subsequence(X1, 400 - 2 * k, k)

    X2 = random_insertion(X, k)
    X2 = random_deletion(X2, k)
    X2 = random_mutation(X2, k)
    X2 = repeat_subsequence(X2, 400 - 2 * k, k)
    X2 = delete_subsequence(X2, 2 * k, k)

    return X1, X2

for k in [10, 20, 30, 40, 50]:
    X1, X2 = generate_variations(X, k)

    start_time_nw = time.time()
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    alignment = aligner.align(X1, X2)
    nw_score = alignment.score
    duration_nw = time.time() - start_time_nw

    start_time_blast = time.time()
    result_handle = NCBIWWW.qblast("blastn", "nt", X1)
    blast_record = NCBIXML.read(result_handle)
    blast_score = max(hsp.score for alignment in blast_record.alignments for hsp in alignment.hsps)
    duration_blast = time.time() - start_time_blast

    print(f"k = {k}")
    print(f"Needleman-Wunsch score: {nw_score}, Time taken: {duration_nw} seconds")
    print(f"BLAST score: {blast_score}, Time taken: {duration_blast} seconds")
    print("-" * 40)


k = 10
Needleman-Wunsch score: 348.0, Time taken: 0.0021347999572753906 seconds
BLAST score: 136.0, Time taken: 66.92744135856628 seconds
----------------------------------------
k = 20
Needleman-Wunsch score: 319.0, Time taken: 0.0019021034240722656 seconds
BLAST score: 149.0, Time taken: 76.45354580879211 seconds
----------------------------------------
k = 30
Needleman-Wunsch score: 298.0, Time taken: 0.0021550655364990234 seconds
BLAST score: 87.0, Time taken: 138.22266912460327 seconds
----------------------------------------
k = 40
Needleman-Wunsch score: 294.0, Time taken: 0.001962900161743164 seconds
BLAST score: 88.0, Time taken: 141.95643520355225 seconds
----------------------------------------
k = 50
Needleman-Wunsch score: 290.0, Time taken: 0.0033791065216064453 seconds
BLAST score: 75.0, Time taken: 79.69526410102844 seconds
----------------------------------------
